# THE GREAT FILE DE-DUPLICATOR
### A solution to my fractured backups
A program of 1000 lines begins with dependency imports


In [6]:
from pathlib import Path
import multiprocess #multiprocessing if running outside of jupyter
from multiprocess import Process, Queue #multiprocessing if running outside of jupyter
import json
from collections import defaultdict
from pprint import pprint, pformat
import time
import logging
from queue import Empty
import os
import pandas as pd

#import pandas as pd
#import zlib
from collections import Counter
from IPython.display import clear_output

print('done')

done


### Enter your parameters
Enter where you want to output your analysis files, what directories you want to analyze for duplicates and which folder names you wish to exclude from the analysis.
Mind the backslashes playing nice with python on windows.

**THE ORDER THE deDupeDirs ARE IN ARE THE PREFERRED ORDER FOR RETENTION IF DRY RUN IS DISABLED! MAKE SURE YOU ARE OK WITH LATER ENTRIES IN THE LIST HAVING DUPLICATE FILES DISABLED!**


In [17]:
dedupeDirs = (r'C:\Users\Ryan\Desktop',
               r'D:\Users\Ryan',
               'G:',
               'F:',
               #'H:\\',
               )
excludeDirs = {'$RECYCLE.BIN',
               'gdpr data',
               'steamapps',
               'Google Dump',
               'Windows',
              }
excludeExtensions = {'.example',    
}
outputDir = Path(r'C:\Users\Ryan\Desktop')

#Assign file retention priority to dedupeDirs based on order
#Early entries in the list will keep their files over later entries with duplicate files
print(dedupeDirs)

('C:\\Users\\Ryan\\Desktop', 'D:\\Users\\Ryan', 'G:', 'F:')


## Find duplicates based on file name
For a given directory listed in "deDupeDirs", get the filenames, last modified dates, and file size. 
End result of a list will be a dict with filename as keys and values that are lists of dicts of file paths and attributes

In [26]:
dupeSearchStart = time.time()
dedupePaths = [Path(x) for x in dedupeDirs]

files = []
dupeDict = defaultdict(list)
for workDir in dedupePaths:
    errors = []
    print(f'running in {workDir}')
    dirDupeSearchStart = time.time()
    for file in workDir.glob('**/*'):
        try:
            if file.is_file() and not excludeDirs.intersection(set(file.parts)) and not excludeExtensions.intersection(set(file.suffixes)):
                fileDict = {'pathObj':file,
                            'path':file.as_posix(),
                            'name':file.name,
                            'size':file.stat().st_size,
                            'mtime':file.stat().st_mtime,
                            }
                dupeDict[file.name].append(fileDict)
                files.append(fileDict)
        except:
                errors.append(file)
            #    raise
    print('len of files parsed = ', len(dupeDict))
    print(f'{len(errors)=}')

    print(f'processed {workDir} in {time.time() - dirDupeSearchStart}s')

uniqueDict = {k:v for k,v in dupeDict.items() if len(v) == 1}
print('number of uniques = ', len(uniqueDict))
dupeDict   = {k:v for k,v in dupeDict.items() if len(v) > 1}
print('number of dupes = ', len(dupeDict))
print(f'dict load completed in {time.time() - dupeSearchStart}s')
print('loading dataframe')
df = pd.DataFrame(files)
print('dupeDataframe loaded')
print(f'df load completed in {time.time() - dupeSearchStart}s')


running in C:\Users\Ryan\Desktop
len of files parsed =  41595
len(errors)=0
processed C:\Users\Ryan\Desktop in 20.11954116821289s
running in D:\Users\Ryan
len of files parsed =  127057
len(errors)=9
processed D:\Users\Ryan in 42.75087833404541s
running in G:
len of files parsed =  127057
len(errors)=0
processed G: in 0.0s
running in F:
len of files parsed =  132049
len(errors)=0
processed F: in 54.5726056098938s
number of uniques =  95809
number of dupes =  36240
dict load completed in 117.52304244041443s
loading dataframe
dupeDataframe loaded
df load completed in 118.02515625953674s


### Save your progress
The above steps could take a while depending on how much you've hoarded, save the output to a JSON file to noodle with at a later date

In [35]:
%%time
errors = []
successCount = 0
for d in dicts:
    for k,li in d.items():
        for file in li:
            try:
                del file['pathObj']
                successCount += 1
            except:
                errors.append(file)
                
                #raise
print(f'{len(errors)=}')
print(f'{successCount=}')
                
with open(outputDir.joinpath('duplicateFiles.json'),'w',encoding='utf8') as f:
    json.dump(dupeDict, f, indent = 1 )
    
with open(outputDir.joinpath('uniqueFiles.json'),'w',encoding='utf8') as f:
    json.dump(uniqueDict, f, indent = 1 )
print(f'done, took {time.time() - writingStart:.2f}s')

df.to_pickle(r'C:\Users\Ryan\Desktop\filelist.pkl')
print('pickled df')

writing to file
len(errors)=228677
successCount=0
done, took 3.38s
pickled df


In [2]:
for d in dupeDict.items():
    print(d)
    break
    
#df.to_csv('out.csv')

NameError: name 'dupeDict' is not defined

In [13]:
%%time
from time import sleep
#for x in range(3):
sleep(1)
x = [x for x in range(19999999)]



Wall time: 3.05 s


### Reload your progress
Reload if you saved to a JSON file.  You'll need to run imports and the parameter code blocks before this.

In [1]:
loadStart = time.time()

df = pd.read_pickle(r'C:\Users\Ryan\Desktop\filelist.pkl')            
            
print(f'done, took {(time.time() - loadStart):.2f}s')

NameError: name 'time' is not defined

### Define functions used in duplicate processing

In [15]:
def mpCrc32(fileName, q, printQueue):
    #have to import within function because of multiprocessing
    from zlib import crc32
    try:
        with open(fileName, 'rb') as fh:
            hash = 0
            while True:
                s = fh.read(65536)
                if not s:
                    break
                hash = crc32(s, hash)
            returnVal = {'file':Path(fileName),'crc32':"%08X" % (hash & 0xFFFFFFFF)}
            #printQueue.put(returnVal)
            q.put(returnVal)
    except Exception as e:
        printQueue.put(f'error on {fileName}, {e}')
        raise

def sizeof_fmt(num, suffix='B'):
    for unit in ['','Ki','Mi','Gi','Ti','Pi','Ei','Zi']:
        if abs(num) < 1024.0:
            return f"{num:3.1f}{unit}{suffix}" 
        num /= 1024.0
    return "{num:.1f}Yi{suffix}"

print('done')

done


ok so I think dataframes are the way to go

get a list of all files

group by name or size or name then size.

run the same crc checks on a group

if the crc checks match, analyze 

group by analysis has an order by clause, keep the top x and have a function for delete with dry run as the base

### Only run on exact duplicates first

In [ ]:
progressTick = 20
progressTick = len(dupeDict.values()) % progressTick
maxProcesses = multiprocess.cpu_count()/2
runningProcesses = []
q = Queue()
printQ = Queue()
crcStart = time.time()

print(f'starting {len(dupeDict)=}')

for dupeGroupCount, kv in enumerate(dupeDict.items()):
    filename,dupeList = kv
    
    if dupeGroupCount % progressTick == 0 or dupeGroupCount == len(dupeDict) + 1:
        clear_output()
        print(f'processing {dupeGroupCount=} of {len(dupeDict)=} for a % complete of {(dupeGroupCount/len(dupeDict)):.2f}')

    #only things in sizeGroup less than  get a CRC
    sizeGroup = defaultdict(list)
    for cnt,d in enumerate(dupeList):
        sizeGroup[d['size']].append(d['path'])
        
    #remove all where we don't have a duplicate  filesize
    #size less than 1000000000 ties to 1 gig
    sizeGroup = {k:v for k,v in sizeGroup.items() if len(v) > 1 and int(k) < 1000000000}
    #pprint(sizeGroup)
    
    for cnt,d in enumerate(dupeList):
        #it would be better to have long running processes that get work through the queue instead of 1 process per file 
        while len(multiprocess.active_children()) == maxProcesses:
            time.sleep(.1)

        print(f"processing {d['path']} {sizeof_fmt(d['size'])}")
        process = multiprocess.Process(target=mpCrc32, args =[d['path'],q, printQ])
        runningProcesses.append(process)
        process.start()
            
    #time.sleep(1)
    
        if cnt > 5:
            time.sleep(1)
            print('breaking for debugging')
            break
            
    if dupeGroupCount > 5:
        #time.sleep(1)
        print('breaking for debugging')
        break
        
print('joining processes')
for p in runningProcesses:
    p.join()

while True:
    try:
        result = q.get(False)  #gotta use False otherwise get is blocking
        name      = result['path'].name
        posixPath = result['path'].as_posix
        found = False
        for d in dupeDict[name]:
            if d['path'] == posixPath:
                found = True
                d.update(result)
                print(d)
                break
        #print('q',result)
    except Empty:
        print('empty')
        raise Exception

    

processing dupeGroupCount=0 of len(dupeDict)=17257 for a % complete of 0.00
processing C:/Users/Ryan/Desktop/desktop.ini 282.0B
processing C:/Users/Ryan/Desktop/Files/Home and Money/Tax/2015 Tax/desktop.ini 136.0B
processing C:/Users/Ryan/Desktop/Files/Music/desktop.ini 440.0B
processing C:/Users/Ryan/Desktop/Files/photos/Old Photos/Trivia Pics/desktop.ini 136.0B
processing C:/Users/Ryan/Desktop/Files/Work/work git/SQL/desktop.ini 110.0B
processing F:/Files/Home and Money/Tax/2015 Tax/desktop.ini 136.0B
processing F:/Files/Music/desktop.ini 440.0B
breaking for debugging
processing C:/Users/Ryan/Desktop/Files/Guide2DataMining.pdf 138.6MiB
processing F:/Files/Guide2DataMining.pdf 138.6MiB
processing C:/Users/Ryan/Desktop/Files/rsync_backup.sh 43.0B
processing F:/Files/rsync_backup.sh 43.0B
processing C:/Users/Ryan/Desktop/Files/Setup.x64.en-US_ProPlusRetail_9WJKN-DG84F-FQR4V-BGDXG-72XKQ_TX_PR_act_1_.exe 1.4MiB
processing F:/Files/Setup.x64.en-US_ProPlusRetail_9WJKN-DG84F-FQR4V-BGDXG-72XK

In [ ]:
startt = time.time()
time.sleep(.1)
print(time.time() - startt)

In [ ]:
    input()
    process = multiprocessing.Process(target=crc32, args =(d['path']))
    runningProcesses.append(process)
    process.start()

    for p in runningProcesses:
        p.join()




    def my_func(arg):
        Q.put('Hello, ' + arg)

    p1 = Process(target=my_func, args=('John',))
    p1.start()
    print(Q.get())
    p1.join()
        
    #fileSizeMatch = Counter([d['size']])
    crcCount = Counter([d['crc32'] for d in dupeList])

    for d in dupeList:
        d['crcDupeCount'] = crcCount[d['crc32']]
    
    dupeList.sort(key=lambda x: x['crcDupeCount'], reverse=True)
    #pprint(dupeList)

    #if cnt + 1 == len(dupeList):
    #Counter
    #pprint(filename)
    #break
#exactDupeDict =
    
print(f'done, took {(time.time() - crcStart):.2f}s')

In [22]:
crcCount['9E997366']

4

In [ ]:
li =[x for x in range(10)]
for cnt,x in enumerate(li):
	print(cnt,x, len(li))

### let's do some analysis by file suffixes

In [ ]:
suffixesCounter = Counter()

for k,dupeList in dupeDict.items(): 
    for file in dupeList:
        #for suffix in file['pathObj'].suffixes:
        suffixesCounter[file['pathObj'].suffix] += 1
suffixesCounter = [(k,v) for k,v in suffixesCounter.items()]
suffixesCounter.sort(key = lambda k: k[1], reverse=True)

#top 20 duplicated 
pprint(suffixesCounter[:20])

### start by figuring out duplication categories
 - same name, same size, same mtime, same hash different dedupeDirs
     - can safely delete one of them based on priority
 - same name, same size, different mtime
     - hash and branch logic if different
 - same name, different size 
     - send to manual review queue, potentially prefer the newer one

In [19]:
if type(dedupeDirs[0]) == str:
    dedupeDirs = [(rootTier,Path(directory).as_posix()) for rootTier,directory in enumerate(dedupeDirs)]

for k,dupeList in dupeDict.items(): 
    for cnt, file in enumerate(dupeList):
        #assign a tier to each identified dupe based on the directory order given in the first user parameters
        for rootTier, dedupeDir in dedupeDirs:
            if file['path'].startswith(dedupeDir):
                file['rootTier'] = rootTier
    dupeList = sorted(dupeList, key=lambda k: (k['rootTier'], len('path')))
    for cnt,file in enumerate(dupeList):
        file['priority'] = cnt 
    pprint(dupeList)

    break
    

[{'mtime': 1607128670.8325891,
  'path': 'C:/Users/Ryan/Desktop/desktop.ini',
  'pathObj': WindowsPath('C:/Users/Ryan/Desktop/desktop.ini'),
  'priority': 0,
  'rootTier': 0,
  'size': 282},
 {'mtime': 1538450652.6097116,
  'path': 'C:/Users/Ryan/Desktop/Files/Home and Money/Tax/2015 Tax/desktop.ini',
  'pathObj': WindowsPath('C:/Users/Ryan/Desktop/Files/Home and Money/Tax/2015 Tax/desktop.ini'),
  'priority': 1,
  'rootTier': 0,
  'size': 136},
 {'mtime': 1577863147.0110621,
  'path': 'C:/Users/Ryan/Desktop/Files/Music/desktop.ini',
  'pathObj': WindowsPath('C:/Users/Ryan/Desktop/Files/Music/desktop.ini'),
  'priority': 2,
  'rootTier': 0,
  'size': 440},
 {'mtime': 1538450652.605711,
  'path': 'C:/Users/Ryan/Desktop/Files/photos/Old Photos/Trivia '
          'Pics/desktop.ini',
  'pathObj': WindowsPath('C:/Users/Ryan/Desktop/Files/photos/Old Photos/Trivia Pics/desktop.ini'),
  'priority': 3,
  'rootTier': 0,
  'size': 136},
 {'mtime': 1616551386.9991388,
  'path': 'C:/Users/Ryan/Desk

### Let's get deleting files
Work out the priority of each file to conslidate everything on one disk first.

Dry run is enabled unless you disable it.

In [3]:
import multiprocess

def double(a):
    return a * 2

def driver_func():
    PROCESSES = 4
    with multiprocess.Pool(PROCESSES) as pool:
        params = [(1, ), (2, ), (3, ), (4, )]
        results = [pool.apply_async(double, p) for p in params]

        for r in results:
            print('\t', r.get())


In [14]:
driver_func()

	 2
	 4
	 6
	 8


In [17]:
from multiprocess import Process, Queue

def f(q):
    q.put('hello world')

q = Queue()
p = Process(target=f, args=[q])
p.start()
print (q.get())
p.join()

hello world


dryRun = True

## TODO:
 - compare by file attributes, basically just size and name
     -  then by hash if same name/size
 - if name/size disagree, put in a conflict csv to review later



Bold 	**bold text**
*italicized text*

Blockquote 	> blockquote
Ordered List 	
1. First item
2. Second item
3. Third item
Unordered List 	
- First item
- Second item
- Third item
Code 	`code`
Horizontal Rule 	---
Link 	
[title](https://www.example.com)
Image 	
![alt text](image.jpg)

### Results 
| Stretch/Untouched | ProbDistribution | Accuracy |
| :- | -: | :-: |
| Stretched | Gaussian | .843

 ```
{
  "firstName": "John",
  "lastName": "Smith",
  "age": 25
}
``` 
footnote 	Here's a sentence with a footnote. [^1]

[^1]: This is the footnote.
Heading ID 
### My Great Heading {#custom-id}
Definition List 	
term
: definition

Task List 	
- [x] Write the press release
- [ ] Update the website

